<a href="https://colab.research.google.com/github/jagjot29/Mini-Project/blob/main/Logistic_Regression.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install streamlit --quiet
!pip install pyngrok==4.1.1 --quiet
from pyngrok import ngrok
!pip install joblib --quiet

In [ ]:
%%writefile app.py
import streamlit as st
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from skimage.io import imread
from skimage.transform import resize
import pickle
from PIL import Image
from sklearn.model_selection import GridSearchCV 
from sklearn import svm
from sklearn.model_selection import train_test_split

datadir = '/content/drive/MyDrive/Minor Project/Images'

categories = ['animals' , 'birds'] # Name the folder with these categories
target = []
flat_data = []
images = []

for i in categories:
  class_num = categories.index(i)
  path = os.path.join(datadir,i)

  for img in os.listdir(path):
    
    img_array = imread(os.path.join(path,img))

    img_resize = resize(img_array,(150,150,3)) # Value gets converted 0-1 from 0-255
    flat_data.append(img_resize.flatten())
    images.append(img_resize)
    target.append(class_num)
    

flat_data = np.array(flat_data)
target = np.array(target)
images = np.array(images)

x_train,x_test,y_train,y_test = train_test_split(flat_data,target,test_size=0.3,random_state=109)
param_grid = [
              {'C':[1,10,100,1000],'kernel':['linear']},
              {'C':[1,10,100,1000],'gamma':[0.001,0.0001],'kernel':['rbf']},
]
svc = svm.SVC(probability=True)
clf = GridSearchCV(svc,param_grid,cv=3)
clf.fit(x_train,y_train)

st.set_option('deprecation.showfileUploaderEncoding', False)
st.title('Image Classifier using Logistic Regression')
st.text('Upload the Image')
pickle.dump(clf,open('img_model.p','wb'))
model = pickle.load(open('img_model.p','rb'))
uploaded_file = st.file_uploader("Choose an image...",type="jpg")
if uploaded_file is not None :
  img = Image.open(uploaded_file)
  st.image(img,caption='Uploaded Image')

  if st.button('PREDICT') :
    st.write('Result...')
    CATEGORIES =['animals' , 'birds']
    flat_data=[]
    img = np.array(img)
    img_resized = resize(img,(150,150,3))
    flat_data.append(img_resized.flatten())
    flat_data = np.array(flat_data)
    y_out = model.predict(flat_data)
    y_out = CATEGORIES[y_out[0]]
    st.title(f' PREDICTED OUTPUT: {y_out}')
    q = model.predict_proba(flat_data)
    for index, item in enumerate(CATEGORIES) :
      st.write(f'{item} : {q[0][index]*100}%')


In [ ]:
!nohup streamlit run app.py &

url = ngrok.connect(port='8501')
url